In [ ]:
import json
import os

from transformers import AutoTokenizer, BartForConditionalGeneration

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# Had 44.9 ROUGE-1
model_path = '/gpfs/slayman/pi/gerstein/xt86/jeffreytan/andrew/taska/confit-train/output-mediqa/4/checkpoint-7776'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path).to('cuda')

gold_summaries = []
predictions = []

with open('New-TaskA-ValidationData.jsonl') as file:
    for line in file:
        json_obj = json.loads(line)
        dialog = '\n'.join(json_obj['dialogue'])
        gold_summary = ' '.join(json_obj['summary'])
        gold_summaries.append(gold_summary)

        inputs_dict = tokenizer(
            dialog, max_length=1024, padding="max_length", truncation=True, return_tensors="pt"
        )
        input_ids = inputs_dict.input_ids.to("cuda")
        attention_mask = inputs_dict.attention_mask.to("cuda")

        output_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=128, num_beams=6)

        prediction = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        print('Dialog: ' + dialog)
        print('Prediction: ' + prediction)
        print('----')
        predictions.append(prediction)


In [9]:
predictions = [l[0] for l in predictions]

In [11]:
open('predictions-1863.txt', 'w').write('\n'.join(predictions))

23733